# Volume Indicators - Apple Stock 2023-2025

Testing volume indicators: OBV, ADOSC.

In [2]:
import sys
from pathlib import Path

import plotly.graph_objects as go
import yfinance as yf
from plotly.subplots import make_subplots

sys.path.insert(0, str(Path("../..").resolve()))

from indicators.volume import calculate_adosc, calculate_obv

In [3]:
# Fetch Apple data
df = yf.download("AAPL", start="2023-01-01", end="2025-10-01", auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.330650,129.079567,122.443165,128.468194,112117500
1,2023-01-04,124.602715,126.870731,123.340517,125.125343,89113600
2,2023-01-05,123.281334,125.993089,123.024955,125.361991,80962700
3,2023-01-06,127.817368,128.478048,123.153152,124.257579,87754700
4,2023-01-09,128.339981,131.554653,128.083602,128.655538,70790800


## 1. OBV - On Balance Volume

Cumulative volume based on price direction. Price up → add volume, price down → subtract volume. OBV rising = accumulation, falling = distribution.

In [4]:
obv, obv_ema = calculate_obv(df, ema_period=55)
df["obv"] = obv
df["obv_ema"] = obv_ema

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price", "Volume", "OBV & OBV EMA")
)

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

# Volume
fig.add_trace(go.Bar(x=df["date"], y=df["volume"], name="Volume", marker_color="lightblue"), row=2, col=1)

# OBV
fig.add_trace(go.Scatter(x=df["date"], y=df["obv"], name="OBV", line={"color": "blue"}), row=3, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["obv_ema"], name="OBV EMA (55)", line={"color": "red", "dash": "dash"}), row=3, col=1
)

fig.update_layout(height=800, title_text="On Balance Volume")
fig.show()

print("OBV = Cumulative volume based on price direction")
print("Price up → Add volume")
print("Price down → Subtract volume")
print("OBV rising = Accumulation")
print("OBV falling = Distribution")

OBV = Cumulative volume based on price direction
Price up → Add volume
Price down → Subtract volume
OBV rising = Accumulation
OBV falling = Distribution


## 2. ADOSC - Chaikin A/D Oscillator

ADOSC = (A/D EMA 3) - (A/D EMA 10). Measures accumulation/distribution momentum. Positive = buying pressure, negative = selling pressure.

In [5]:
df["adosc"] = calculate_adosc(df, fastperiod=3, slowperiod=10)

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price", "Volume", "ADOSC")
)

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

# Volume
fig.add_trace(go.Bar(x=df["date"], y=df["volume"], name="Volume", marker_color="lightblue"), row=2, col=1)

# ADOSC
fig.add_trace(
    go.Bar(
        x=df["date"], y=df["adosc"], name="ADOSC", marker_color=["green" if val >= 0 else "red" for val in df["adosc"]]
    ),
    row=3,
    col=1,
)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=3, col=1)

fig.update_layout(height=800, title_text="Chaikin A/D Oscillator")
fig.show()

print("ADOSC = (A/D EMA 3) - (A/D EMA 10)")
print("Measures accumulation/distribution momentum")
print("Positive = Buying pressure")
print("Negative = Selling pressure")
print(f"\nADOSC mean: {df['adosc'].mean():.2e}")

ADOSC = (A/D EMA 3) - (A/D EMA 10)
Measures accumulation/distribution momentum
Positive = Buying pressure
Negative = Selling pressure

ADOSC mean: 1.28e+07


## OBV vs Price Divergence Analysis

In [6]:
# Normalize price and OBV for comparison
from sklearn.preprocessing import MinMaxScaler

scaler_price = MinMaxScaler()
scaler_obv = MinMaxScaler()

df["close_norm"] = scaler_price.fit_transform(df[["close"]])
df["obv_norm"] = scaler_obv.fit_transform(df[["obv"]])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df["date"], y=df["close_norm"], name="Price (normalized)", line={"color": "black", "width": 2})
)
fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["obv_norm"], name="OBV (normalized)", line={"color": "blue", "width": 2, "dash": "dash"}
    )
)

fig.update_layout(
    height=600, title_text="Price vs OBV Divergence Analysis", xaxis_title="Date", yaxis_title="Normalized Value (0-1)"
)
fig.show()

print("When price and OBV diverge, potential reversal ahead")
print("Price ↑ + OBV ↓ = Bearish divergence (distribution)")
print("Price ↓ + OBV ↑ = Bullish divergence (accumulation)")

When price and OBV diverge, potential reversal ahead
Price ↑ + OBV ↓ = Bearish divergence (distribution)
Price ↓ + OBV ↑ = Bullish divergence (accumulation)


## Combined View - All Volume Indicators

In [7]:
fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=("Price", "Volume", "OBV & OBV EMA", "ADOSC"),
)

# Row 1: Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 2}), row=1, col=1)

# Row 2: Volume
fig.add_trace(
    go.Bar(x=df["date"], y=df["volume"], name="Volume", marker_color="lightblue", showlegend=False), row=2, col=1
)

# Row 3: OBV
fig.add_trace(go.Scatter(x=df["date"], y=df["obv"], name="OBV", line={"color": "blue", "width": 1.5}), row=3, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["obv_ema"], name="OBV EMA", line={"color": "red", "dash": "dash", "width": 1.5}),
    row=3,
    col=1,
)

# Row 4: ADOSC
fig.add_trace(
    go.Bar(
        x=df["date"],
        y=df["adosc"],
        name="ADOSC",
        marker_color=["green" if val >= 0 else "red" for val in df["adosc"]],
        showlegend=False,
    ),
    row=4,
    col=1,
)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=4, col=1)

fig.update_layout(height=1000, title_text="Volume Indicators - Complete Overview")
fig.show()

print("\n=== Volume Summary ===")
print(f"OBV trend: {'Rising' if df['obv'].iloc[-1] > df['obv'].iloc[0] else 'Falling'}")
print(f"ADOSC mean: {df['adosc'].mean():.2e}")
print(f"ADOSC positive periods: {(df['adosc'] > 0).sum()} days ({(df['adosc'] > 0).sum() / len(df) * 100:.1f}%)")
print(f"ADOSC negative periods: {(df['adosc'] < 0).sum()} days ({(df['adosc'] < 0).sum() / len(df) * 100:.1f}%)")
print("\nVolume confirms price moves. Divergences signal potential reversals.")


=== Volume Summary ===
OBV trend: Rising
ADOSC mean: 1.28e+07
ADOSC positive periods: 460 days (66.9%)
ADOSC negative periods: 227 days (33.0%)

Volume confirms price moves. Divergences signal potential reversals.
